In [1]:
import pandas as pd
from collections import defaultdict
import nltk
from collections import Counter
import re
import math


In [2]:
# Cargar el archivo de entrenamiento
with open("C:/Users/santi/Documents/Maestría CDAA/Datos aprendaut/Chat de WhatsApp con Anita17_1_23-5_5_23.txt", "r") as file:
    chat = file.read()

In [3]:
medio = len(chat)//2
chat = chat[0:medio]

In [4]:
# Procesamiento de del chat para obtener una lista de palabras
lines = chat.split('\n')
words = []

for line in lines:
    if ":" in line:
        message = line.rsplit(":", 1)[-1].strip()
        message_clean = re.findall(r'[\wáéíóúÁÉÍÓÚñÑ]+', message)  
        words.extend(word.lower() for word in message_clean) # conservamos las palabras pero en minusculas
        

In [5]:
def frases_de_horizonte(horizonte, words):
    '''
    Devuelve un diccionario con un key por palabra del vocabulario y valores las frases de largo horizonte que terminan en la palabra
    '''
    iter = list(set(words))
    frases= {}
    for target in iter: #Tomo las palabras para las que quiero las probab condicionales
        # Length of sublists you want to find
        n = horizonte
        # listas de palabras con la palabra target
        frases[target] = [words[max(i-n,0):i] for i in range(len(words)) if target == words[i]]
    return frases



In [6]:
def cuantos(b, a, frases):
    '''
    Cuenta la cantidad de frases que terminan en a tienen a b
    '''
    frases_con_b = 0
    if a in frases:
        for words in frases[a]:
            if b in words:
                frases_con_b += 1
    return frases_con_b

In [7]:
#Diccionario con las probabilidades a priori de las palabras
def Priori(words):
    '''
    Calcula un diccionario con las probabilidades a priori de cada palabra
    '''
    P = {}
    for palabra in list(set(words)):
        P[palabra] = words.count(palabra)/len(words) #cantidad de ocurrencias de la palabra en todas las instancias
    return P

In [8]:
def Probabilidades_condicionales(words, horizonte, m):
    '''
    Calcula un diccionario con las probabilidades condicionales de cada palabra de estar en una frase de largo horizonte con cada otra palabra
    '''
    PD = {} #Diccionario con las probabilidades condicoinales m-estimador

    frases = frases_de_horizonte(horizonte, words)

    pal_unicas = list(set(words))

    for a in pal_unicas: # P( _ | a)
        PD[a]={}
        itera = pal_unicas.copy()
        itera.remove(a)
        for b in itera: # P( b | a )
            e = cuantos(b,a,frases) #frases con la palabra b que terminan en a
            p = 1/len(pal_unicas) #probabilidad de una palabra
            n = len(frases[a]) #frases que terminan con a
            PD[a][b]= (e + m*p)/(n+m)
    return PD

In [10]:
horizonte = 2
m = 2
PD = Probabilidades_condicionales(words, horizonte, m)
P = Priori(words)

In [28]:
"cómo" in PD["cómo"]

False

In [27]:
P_nada = 0.000000001
PD["cómo"].get("cómo", P_nada)

1e-09

In [43]:
def recomendacion_bayesiana(frase):
    D = frase   # Frase que será ingresada
    
    P_nada = 0.000000001  # valor pequeño por si la palabra no aparece en el diccionario
    h_MAP = "" 
    p_MAP = float('-inf') ##variable que se utilizará para almacenar la probabilidad más alta encontrada hasta ahora.

    for h in P: # me fijo en cada palabra posible
        prob = math.log(P[h])
        #print("------------ \n P(",h,")","{:.3f}".format(prob), " + ")
        for d in D[-horizonte:]: #P(sugerencia) P(pal1|sugerencia) ... P(paln|sugerencia)
            print("P(",d,"|",h, ")","{:.3f}".format(math.log(PD[h].get(d, P_nada))), " + ")
            prob = prob + math.log(PD[h].get(d, P_nada))
        if prob > p_MAP:
            h_MAP, p_MAP = h, prob
            #print("Probabilidad de ", h, " = ", prob)
    #print("--------\n --->",h_MAP, p_MAP, "<---\n -------------------")
    return(h_MAP)


In [44]:
##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))





Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
------------ 
 P( vos ) -5.989  + 
P( ani | vos ) -14.025  + 
Probabilidad de  vos  =  -20.014225370775264
------------ 
 P( presionada ) -10.872  + 
P( ani | presionada ) -9.828  + 
------------ 
 P( digass ) -11.565  + 
P( ani | digass ) -9.540  + 
------------ 
 P( convenga ) -11.565  + 
P( ani | convenga ) -9.540  + 
------------ 
 P( hablabamos ) -11.565  + 
P( ani | hablabamos ) -9.540  + 
------------ 
 P( unos ) -7.902  + 
P( ani | unos ) -12.155  + 
------------ 
 P( anterior ) -9.956  + 
P( ani | anterior ) -10.387  + 
------------ 
 P( speak ) -11.565  + 
P( ani | speak ) -9.540  + 
------------ 
 P( cami ) -9.368  + 
P( ani | cami ) -10.839  + 
------------ 
 P( dramático ) -11.565  + 
P( ani | dramático ) -9.540  + 
------------ 
 P( fuee